In [ ]:
# Cell 1
from google.colab import drive
drive.mount('/content/drive')

# Cell 2
%cd /content/drive/MyDrive/GT_Semester3/Deep\ RL\ \(CS\ 8803\ DRL\)/Project/openvla

Mounted at /content/drive
/content/drive/MyDrive/GT_Semester3/Deep RL (CS 8803 DRL)/Project/openvla


In [ ]:
!pip install -e .


Obtaining file:///content/drive/MyDrive/GT_Semester3/Deep%20RL%20%28CS%208803%20DRL%29/Project/openvla
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/moojink/dlimp_openvla to /tmp/pip-install-0smuf4x2/dlimp_1e143c636d71419d8b50bbdea48e3c49
  Running command git clone --filter=blob:none --quiet https://github.com/moojink/dlimp_openvla /tmp/pip-install-0smuf4x2/dlimp_1e143c636d71419d8b50bbdea48e3c49
  Resolved https://github.com/moojink/dlimp_openvla to commit 040105d256bd28866cc6620621a3d5f7b6b91b46
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

# OpenVLA Training?

In [ ]:
# !git clone https://github.com/openvla/openvla.git

In [ ]:
%pwd

'/content/drive/MyDrive/GT_Semester3/Deep RL (CS 8803 DRL)/Project/openvla'

In [ ]:
import tensorflow_datasets as tfds
print(tfds.core.constants.DATA_DIR)


/root/tensorflow_datasets


In [ ]:
%cd ../Dataset/

/content/drive/MyDrive/GT_Semester3/Deep RL (CS 8803 DRL)/Project/Dataset


In [ ]:
import json

# Load the JSON file
with open('matched_commands_with_coordinates.json', 'r') as f:
    data = json.load(f)

# Split the data into texts and labels
texts = [item['command'] for item in data]
labels_xy = [tuple(item['coordinates']) for item in data]

# Print the results
print("Texts:", len(texts), texts)
print("Labels:", len(labels_xy), labels_xy)


Texts: 1000 ['Align your position with the ball', 'Head over to the pineapple', 'Engage approach to the pineapple', 'Position yourself near the dog', 'Zero in on the cat', 'Proceed toward the ball', "Travel to the cat's position", 'Make your way to the cow', "Reach the ball's location", 'Engage approach to the pineapple', 'Focus your movement on the cow', 'Engage approach to the dog', 'Step forward to the pineapple', 'Go closer to the dog', "Target the cow's location", 'Align your position with the cat', 'Step forward to the cow', 'Direct yourself toward the dog', 'Proceed toward the pineapple', 'Engage approach to the pineapple', 'Adjust your path to reach the cat', "Progress toward the cow's vicinity", 'Advance to the ball', "Approach the ball's location", 'Head over to the dog', 'Relocate to the pineapple', 'Head over to the dog', 'Navigate to the dog', "Target the pineapple's location", "Progress toward the cow's vicinity", 'Move toward the pineapple', "Reach the ball's location", 

In [ ]:
image_paths = [
    f"image_{i}.png"
    for i in range(len(texts))
]
print(image_paths[:5])
print(len(image_paths))

['image_0.png', 'image_1.png', 'image_2.png', 'image_3.png', 'image_4.png']
1000


In [ ]:
%cd ../openvla/

/content/drive/MyDrive/GT_Semester3/Deep RL (CS 8803 DRL)/Project/openvla


In [ ]:
# create_example_data.py
import numpy as np
import os
import tqdm
from PIL import Image

N_EXAMPLES = 1000
os.makedirs('data/train', exist_ok=True)

image_directory = "../Dataset/images_0/"

# Example placeholders: replace these with your actual data.
# image_paths = ['path_to_image_0.jpg', 'path_to_image_1.jpg', ...]
# texts = ['some text for 0', 'some text for 1', ...]
# labels_xy = [(x0, y0), (x1, y1), ...] # each an (x,y) pair.


def normalize(actions, min_val=-5, max_val=5):
    """
    Normalizes action values to the range [0, 1].

    Args:
        actions (np.ndarray): Array of action values with shape (num_samples, num_actions).
        min_val (float): Minimum value of the original range.
        max_val (float): Maximum value of the original range.

    Returns:
        np.ndarray: Normalized action values.
    """
    return (actions - min_val) / (max_val - min_val)


# Make sure lengths match: len(image_paths) == len(texts) == len(labels_xy) == N_EXAMPLES


for i in tqdm.tqdm(range(N_EXAMPLES)):
    # Load and convert image to the correct shape and dtype
    # We assume the image is already 1280(width) x 960(height) RGB.
    # If not, you may need to resize or rotate the image accordingly.

    # Correctly construct the image path
    img_path = os.path.join(image_directory, image_paths[i])

    img = Image.open(image_directory + image_paths[i]).convert('RGB')
    # Ensure shape [960, 1280, 3]
    img = np.asarray(img, dtype=np.uint8)


    x, y = labels_xy[i]

    # Normalize x and y
    x_norm = normalize(x)
    y_norm = normalize(y)


    # Append 5 dimensions of zeros to the existing (x, y) values
    label_with_zeros = np.append([x_norm, y_norm], [0, 0, 0, 0, 0]).astype(np.float32)

    # Create a single-step episode
    episode = [{
        'image': img,
        'language_instruction': texts[i],
        # Updated label creation
        'label': label_with_zeros,
    }]

    # Save as a .npy file
    np.save(f'data/train/episode_{i}.npy', episode)

print("Successfully created the dataset!")


100%|██████████| 1000/1000 [10:26<00:00,  1.60it/s]

Successfully created the dataset!


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# example_dataset_dataset_builder.py
from typing import Iterator, Tuple, Any
import glob
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

class CustomDataset(tfds.core.GeneratorBasedBuilder):
    """DatasetBuilder for single-step episodes with image, text, and (x,y) label."""

    VERSION = tfds.core.Version('1.0.0')
    RELEASE_NOTES = {
        '1.0.0': 'Initial release.'
    }

    def __init__(self, *args, **kwargs):
      super().__init__(*args, **kwargs)
      self._embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

    def _info(self) -> tfds.core.DatasetInfo:
        return self.dataset_info_from_configs(
            features=tfds.features.FeaturesDict({
                'steps': tfds.features.Dataset({
                    'observation': tfds.features.FeaturesDict({
                        'image': tfds.features.Image(
                            shape=(960, 1280, 3),
                            dtype=np.uint8,
                            encoding_format='png',
                            doc='Main camera RGB observation (960x1280x3).'
                        ),
                    }),
                    'action': tfds.features.Tensor(
                        shape=(7,),
                        dtype=np.float32,
                        doc='Robot action, consists of [7x joint velocities, '
                            '2x gripper velocities, 1x terminate episode].',
                    ),
                    'discount': tfds.features.Scalar(
                        dtype=np.float32,
                        doc='Discount factor, set to 1.0 since no RL sequence.'
                    ),
                    'reward': tfds.features.Scalar(
                        dtype=np.float32,
                        doc='Reward, set to 0.0 for non-RL data.'
                    ),
                    'is_first': tfds.features.Scalar(
                        dtype=np.bool_,
                        doc='True since only one step per episode.'
                    ),
                    'is_last': tfds.features.Scalar(
                        dtype=np.bool_,
                        doc='True since only one step per episode.'
                    ),
                    'is_terminal': tfds.features.Scalar(
                        dtype=np.bool_,
                        doc='True since only one step per episode.'
                    ),
                    'language_instruction': tfds.features.Text(
                        doc='Language Instruction.'
                    ),
                    'language_embedding': tfds.features.Tensor(
                        shape=(512,),
                        dtype=np.float32,
                        doc='Kona language embedding. '
                            'See https://tfhub.dev/google/universal-sentence-encoder-large/5'
                    ),
                }),
                'episode_metadata': tfds.features.FeaturesDict({
                    'file_path': tfds.features.Text(
                        doc='Path to the original data file.'
                    )
                }),
            })
        )

    def _split_generators(self, dl_manager: tfds.download.DownloadManager):
        # If you have separate val/test splits, you can also add them here.
        return {
            'train': self._generate_examples(path='data/train/episode_*.npy'),
            # 'val': self._generate_examples(path='data/val/episode_*.npy'),
        }

    # def _generate_examples(self, path) -> Iterator[Tuple[str, Any]]:
    #     episode_paths = glob.glob(path)

    #     for episode_path in episode_paths:
    #         data = np.load(episode_path, allow_pickle=True)
    #         # data is a single-step episode: a list with one dict: {'image', 'text', 'label'}

    #         step = data[0]
    #         episode = [{
    #             'observation': {
    #                 'image': step['image'],
    #                 'text': step['text'],
    #                 'label': step['label'],
    #             },
    #             'discount': 1.0,
    #             'reward': 0.0,
    #             'is_first': True,
    #             'is_last': True,
    #             'is_terminal': True,

    #         }]

    #         sample = {
    #             'steps': episode,
    #             'episode_metadata': {
    #                 'file_path': episode_path
    #             }
    #         }

    #         yield episode_path, sample

    def _generate_examples(self, path) -> Iterator[Tuple[str, Any]]:
        """Generator of examples for each split."""

        def _parse_example(episode_path):
            # load raw data --> this should change for your dataset
            data = np.load(episode_path, allow_pickle=True)     # this is a list of dicts in our case

            # assemble episode --> here we're assuming demos so we set reward to 1 at the end
            episode = []
            for i, step in enumerate(data):
                # compute Kona language embedding
                language_embedding = self._embed([step['language_instruction']])[0].numpy()

                episode.append({
                    'observation': {
                        'image': step['image'],
                        # 'wrist_image': None,
                        # 'state': None,
                        # 'text': step['text'],
                        # 'label': step['label'],
                    },
                    'action': step['label'],
                    'discount': 1.0,
                    'reward': 1,
                    'is_first': True,
                    'is_last': True,
                    'is_terminal': True,
                    'language_instruction': step['language_instruction'],
                    'language_embedding': language_embedding,
                })

            # create output data sample
            sample = {
                'steps': episode,
                'episode_metadata': {
                    'file_path': episode_path
                }
            }

            # if you want to skip an example for whatever reason, simply return None
            return episode_path, sample

        # create list of all examples
        episode_paths = glob.glob(path)

        # for smallish datasets, use single-thread parsing
        for sample in episode_paths:
            yield _parse_example(sample)

        # for large datasets use beam to parallelize data parsing (this will have initialization overhead)
        # beam = tfds.core.lazy_imports.apache_beam
        # return (
        #         beam.Create(episode_paths)
        #         | beam.Map(_parse_example)
        # )




In [ ]:
%pwd

'/content/drive/MyDrive/GT_Semester3/Deep RL (CS 8803 DRL)/Project/openvla'

In [ ]:
!tfds build --datasets=custom_dataset

INFO[build.py]: Loading dataset custom_dataset from path: /content/drive/MyDrive/GT_Semester3/Deep RL (CS 8803 DRL)/Project/openvla/custom_dataset/custom_dataset_dataset_builder.py
2024-12-09 02:22:35.201404: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-09 02:22:35.201478: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-09 02:22:35.202651: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-09 02:22:36.591075: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO[utils.py]: NumExpr defaulting to 12 threads.
2024-12-09

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
from custom_dataset_dataset_builder import CustomDataset
import tensorflow_datasets as tfds

builder = CustomDataset()
builder.download_and_prepare()  # This step generates TFRecords

import os

tfds_dir = os.path.expanduser('~/tensorflow_datasets/custom_dataset/1.0.0/')
print("Contents of the TFDS directory:")
print(os.listdir(tfds_dir))

ds = builder.as_dataset(split='train', as_supervised=False)

for sample in ds.take(1):
    print(sample)

Contents of the TFDS directory:
['dataset_info.json', 'features.json', 'custom_dataset-train.tfrecord-00000-of-00001']
{'episode_metadata': {'file_path': <tf.Tensor: shape=(), dtype=string, numpy=b'data/train/episode_734.npy'>}, 'steps': <_VariantDataset element_spec={'action': TensorSpec(shape=(7,), dtype=tf.float32, name=None), 'discount': TensorSpec(shape=(), dtype=tf.float32, name=None), 'is_first': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_last': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_terminal': TensorSpec(shape=(), dtype=tf.bool, name=None), 'language_embedding': TensorSpec(shape=(512,), dtype=tf.float32, name=None), 'language_instruction': TensorSpec(shape=(), dtype=tf.string, name=None), 'observation': {'image': TensorSpec(shape=(960, 1280, 3), dtype=tf.uint8, name=None)}, 'reward': TensorSpec(shape=(), dtype=tf.float32, name=None)}>}


In [ ]:
%pwd

'/content/drive/MyDrive/GT_Semester3/Deep RL (CS 8803 DRL)/Project/openvla/datasets'

In [ ]:
# %mkdir custom_dataset
# %mv ~/tensorflow_datasets/custom_dataset/1.0.0/* ./custom_dataset/

In [ ]:
%ls ~/tensorflow_datasets/custom_dataset/1.0.0/


custom_dataset-train.tfrecord-00000-of-00001  dataset_info.json  features.json


In [ ]:
from pathlib import Path

Path("hello")

PosixPath('hello')

In [ ]:
%cd ..

/content/drive/MyDrive/GT_Semester3/Deep RL (CS 8803 DRL)/Project/openvla


In [ ]:
!torchrun --standalone --nnodes 1 --nproc-per-node 1 vla-scripts/finetune.py \
  --vla_path "openvla/openvla-7b" \
  --data_root_dir "datasets/" \
  --dataset_name "custom_dataset" \
  --run_root_dir "runs"  \
  --adapter_tmp_dir "adapter-tmp" \
  --lora_rank 32 \
  --batch_size 4 \
  --grad_accumulation_steps 1 \
  --learning_rate 5e-4 \
  --image_aug False \
  --wandb_project "openvla_finetuning" \
  --wandb_entity "myusername" \
  --save_steps 100


2024-12-08 22:56:53.625447: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-08 22:56:53.625578: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-08 22:56:53.629001: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-08 22:56:54.819415: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Fine-tuning OpenVLA Model `openvla/openvla-7b` on `custom_dataset`
Loading checkpoint shards: 100% 3/3 [00:00<00:00,  3.11it/s]
trainable params: 110,828,288 || all params: 7,652,065,472 || trainable%: 1.4483
12/08 [22:57:28] INFO     | >> [

In [ ]:
!ls -lh ~/tensorflow_datasets/custom_dataset/1.0.0/


total 29M
-rw-r--r-- 1 root root  29M Dec  8 20:11 custom_dataset-train.tfrecord-00000-of-00001
-rw-r--r-- 1 root root  38K Dec  8 20:11 dataset_info.json
-rw-r--r-- 1 root root 7.1K Dec  8 20:11 features.json


In [ ]:
!torchrun --standalone --nnodes 1 --nproc-per-node 1 vla-scripts/finetune.py \
  --vla_path "openvla/openvla-7b" \
  --data_root_dir "datasets/" \
  --dataset_name "custom_dataset" \
  --run_root_dir "runs"  \
  --adapter_tmp_dir "adapter-tmp" \
  --lora_rank 32 \
  --batch_size 4 \
  --grad_accumulation_steps 1 \
  --learning_rate 5e-4 \
  --image_aug False \
  --wandb_project "openvla_finetuning" \
  --wandb_entity "myusername" \
  --save_steps 100


2024-12-09 02:32:45.556486: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-09 02:32:45.556537: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-09 02:32:45.557779: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-09 02:32:46.683270: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Fine-tuning OpenVLA Model `openvla/openvla-7b` on `custom_dataset`
Loading checkpoint shards: 100% 3/3 [00:00<00:00,  3.36it/s]
trainable params: 110,828,288 || all params: 7,652,065,472 || trainable%: 1.4483
12/09 [02:33:19] INFO     | >> [